In [ ]:
f63 = [
        "userHost_comments_15m",
        "liveHost_distinctCohostedUsers_7_DAY",
        "userHost_shares_15m",
        "recentEvents_x_likes_unbounded",
        "liveUser_livesViewedInHomeFeed_30_DAY",
        "liveUser_cheersSpent_30_DAY",
        "recentEventsUser_u_th_creators_frac_unbounded",
        "recentEventsUser_u_likes_frac_unbounded",
        "recentEvents_x_requests_1_last_unbounded",
        "recentEvents_x_th_lives_frac_unbounded",
        "userHost_dt_shared_live_7d",
        "recentEvents_x_comments_per_live_unbounded",
        "recentEventsUser_u_num_creators_th_unbounded",
        "userHost_dt_like_6h",
        "userHost_dt_shared_live_6h",
        "sv_user_creator_vplay_7d",
        "recentEventsUser_u_timespent_unbounded",
        "recentEventsUser_u_requests_per_live_unbounded",
        "userHost_dt_shared_live_30d",
        "userHost_likes_15m",
        "recentEvents_x_cohost_1_last_unbounded",
        "recentEvents_x_ts_2_last_unbounded",
        "user_realtime_follow_bias",
        "recentEvents_x_comments_1_last_unbounded",
        "userHost_shares_30d",
        "userHost_shares_7d",
        "userHost_dt_commented_live_30d",
        "svUser_vplay_main_feed_v2_30_DAY",
        "recentEvents_x_shares_1_last_unbounded",
        "dt_cohost_req_4m",
        "recentEvents_followsHost_unbounded",
        "live_livestreamVideoPlay_4_MINUTE",
        "liveUser_distinctCommentedHosts_15_MINUTE",
        "userHost_likes_1h",
        "recentEventsUser_u_comments_unbounded",
        "userHost_dt_commented_live_6h",
        "svUser_vplay_3s_v2_30_DAY",
        "userHost_dt_commented_live_1d",
        "userHost_comments_30d",
        "recentEvents_x_likes_1_last_unbounded",
        "recentEvents_x_comments_unbounded",
        "cohost_req_4m",
        "recentEvents_num_lives_unbounded",
        "recentEventsUser_u_timespent_per_live_unbounded",
        "liveUser_shares_30_DAY",
        "recentEventsUser_u_comments_per_live_unbounded",
        "recentEventsUser_u_num_lives_unbounded",
        "sv_user_creator_vplay_1d",
        "liveUser_distinctCommentedLivestreams_30_DAY",
        "recentEvents_x_timespent_per_live_unbounded",
        "recentEvents_x_ts_1_last_unbounded",
        "live_livestreamVideoPlay_30_MINUTE",
        "userHost_comments_7d",
        "userHost_likes_7d",
        "liveUser_distinctCommentedLivestreams_15_MINUTE",
        "liveUser_distinctCommentedHosts_30_DAY",
        "follow_2m",
        "act_vc",
        "liveUser_comments_30_DAY",
        "sv_user_creator_vplay_unbounded",
        "recentEventsUser_u_num_creators_unbounded",
        "dt_commenter_2m",
        "liveHost_distinctCohostedUsers_1_DAY"
    ]

In [ ]:
import gc
import pandas as pd
import os
import time
from tqdm.notebook import tqdm
import numpy as np
import json
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import json
import polars as pl

In [ ]:
%%time
train = pl.read_parquet("data/train_sampled.parquet")
val = pl.read_parquet("data/val_sampled.parquet")

# train = pl.read_parquet("data/train.parquet")
# val = pl.read_parquet("data/val.parquet")


In [ ]:
len(train), len(val)

In [ ]:
# scaling optional
from sklearn.preprocessing import MinMaxScaler, StandardScaler
min_max_scaler = MinMaxScaler()
train[f63] = min_max_scaler.fit_transform(train[f63])
val[f63] = min_max_scaler.transform(val[f63])

In [ ]:
with open('data/user_id_mapping.json', 'r') as json_file:
    user_id_mapping = json.load(json_file)

In [ ]:
train = train.with_columns( 
                pl.col('userId')
                .map_dict(user_id_mapping) # if x in user_id_mapping else oov_index_user)
                .alias("userIndex"))
val = val.with_columns( 
                pl.col('userId')
                .map_dict(user_id_mapping)# if x in user_id_mapping else oov_index_user)
                .alias("userIndex"))

In [ ]:
users_tr = train["userId"].unique()
users_val = val["userId"].unique()

In [ ]:
common_users = list(set(users_tr.to_list()) | set(users_val.to_list()))

In [ ]:
len(users_tr), len(common_users), len(users_tr)/len(common_users)

In [ ]:
user_id_mapping = {user_id: idx+1 for idx, user_id in enumerate(users_tr)}
oov_index_user = 0

In [ ]:
len(user_id_mapping), len(train.columns), oov_index_user

train = train.with_columns( 
                pl.col('userId')
                .apply(lambda x: user_id_mapping[x] if x in user_id_mapping else oov_index_user)
                .alias("userIndex"))
val = val.with_columns( 
                pl.col('userId')
                .apply(lambda x: user_id_mapping[x] if x in user_id_mapping else oov_index_user)
                .alias("userIndex"))


In [ ]:
val.filter(pl.col("userIndex") == 0)

In [ ]:
61084/len(val)

In [ ]:
train.filter(pl.col("userIndex") == 0)

## sampling??

In [ ]:
# done for oov_index_user = 0  for all ids not in train but in val
len(train)

In [ ]:
train["sampled"] = False

In [ ]:
num_samples = int(0.1 * len(users_tr))
sampled_user_ids = np.random.choice(users_tr, size=num_samples, replace=False)

In [ ]:
sampled_df = train[train["userId"].isin(sampled_user_ids)].reset_index(drop=True)
sampled_df.loc[sampled_df["userId"].isin(sampled_user_ids), "sampled_user"] = True

In [ ]:
train = pd.concat([train, sampled_df], ignore_index=True)
seed = 42
train = train.sample(frac=1, random_state=seed)

In [ ]:
len(train)

# model #

In [ ]:
from transformers import AutoTokenizer , AutoModelForMaskedLM
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import DistilBertModel

In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch
import torch.optim as optim
import random

In [ ]:
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [ ]:
input_cols = f63

In [ ]:
label_cols = [
#         "label",
#           "label_like",
#           "label_share",
#           "label_cmt",
#           "label_cheer",
#           "label_cohost",
#           "label_req",
          "label_ts",
#           "label_qscan",
#           "label_logts"
        ]

In [ ]:
class rankerDataset(Dataset):
    def __init__(self, df, input_cols, label_cols, tokenizer):
        self.label_cols = label_cols
        self.tokenizer = tokenizer
        self.max_sequence_length = MAX_SEQ_LEN
        
        self.userIndices = df['userIndex'].values 
        self.counterf = df[f63].values
        self.label = df["label_ts"].values
        self.text = df["cmt"].values
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        tokens = self.tokenizer(self.text[idx], padding='max_length', truncation=True, max_length=self.max_sequence_length, return_tensors="pt")
        
        return (
                 tokens["input_ids"],
                 tokens["attention_mask"],
                 torch.tensor(self.userIndices[idx]),
                 torch.tensor(self.counterf[idx]),
                 self.label[idx]
        )

In [ ]:
tokenizer

In [ ]:
%%time
train = train.to_pandas()
val = val.to_pandas()

In [ ]:
MAX_SEQ_LEN = 256
BATCH_SIZE = 128

In [ ]:
trDataset = rankerDataset(train, f63, label_cols, tokenizer)
train_dataloader = DataLoader(trDataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

In [ ]:
valDataset = rankerDataset(val, f63, label_cols, tokenizer)
val_dataloader = DataLoader(valDataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

In [ ]:
len(train), len(val), len(train_dataloader), len(val_dataloader)

In [ ]:
class CommentV0(nn.Module):
    def __init__(self, bert_model_name, num_users, embedding_dim_users):
        super(CommentV0, self).__init__()
#         self.bert = BertModel.from_pretrained(bert_model_name)
#         self.bert = AutoModelForSequenceClassification.from_pretrained(bert_model_name)
#         self.bert = model.distilbert

        self.bert = DistilBertModel.from_pretrained(bert_model_name)
        bert_cls_size = self.bert.config.hidden_size

#         for name, param in self.bert.named_parameters():
# #             if "layer.0" in name or "layer.1" in name or "layer.2" in name or "layer.3" in name:
#             if  "layer.5" in name :
#                 print(name)
#                 continue
#             param.requires_grad = False
    
#         final_size = 32 + bert_cls_size + embedding_dim_users
        
        self.user_embedding = nn.Embedding(num_users, embedding_dim_users)
        
        
        self.fc1 = nn.Linear(63 + embedding_dim_users + bert_cls_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)

        self.output_layer = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.5) 
    
        
    def forward(self, input_ids, attention_mask, userId, counterf):
        user_embedding = self.user_embedding(userId)
        bert_embedding = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        x = torch.cat((counterf, user_embedding, bert_embedding), dim=1)
        x = self.dropout(nn.ReLU()(self.bn1(self.fc1(x))))
        x = self.dropout(nn.ReLU()(self.bn2(self.fc2(x))))
        x = self.dropout(nn.ReLU()(self.bn3(self.fc3(x))))

        output = self.output_layer(x)
        
        return output

In [ ]:
class CommentV0(nn.Module):
    def __init__(self, bert_model_name, num_users, embedding_dim_users):
        super(CommentV0, self).__init__()
        self.bert = DistilBertModel.from_pretrained(bert_model_name)
        bert_cls_size = self.bert.config.hidden_size
 
        self.user_embedding = nn.Embedding(num_users, embedding_dim_users)

        self.final = nn.Sequential(
                            nn.Linear(831, 512),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(512, 128),
                            nn.ReLU(),
                            nn.Dropout(0.4),
                            nn.Linear(128, 64),
                            nn.ReLU(),
                            nn.Linear(64, 1)  
                        )
        
        
    def forward(self, input_ids, attention_mask, userId, counterf):
        e1 = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :] 
        e2 = self.user_embedding(userId)
        
        concat = torch.cat((e1, counterf), dim=1)
        out = self.final(concat)
        
        return out

In [ ]:
class CommentV0(nn.Module):
    def __init__(self, bert_model_name, num_users, embedding_dim_users):
        super(CommentV0, self).__init__()
        self.bert = DistilBertModel.from_pretrained(bert_model_name)
        bert_cls_size = self.bert.config.hidden_size

        self.user_embedding = nn.Embedding(num_users, embedding_dim_users)
        
        self.counterf_nw = nn.Sequential(
                                nn.Linear(63, 128),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(128, 32)
                            )
#         self.bert_nw = nn.Linear(768, 32)
        self.bert_nw = nn.Sequential(
                                nn.Linear(768, 128),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(128, 32)
                            )
        
        self.final = nn.Sequential(
                                nn.Linear(32, 64),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(64, 1)
                            )
        
        
    def forward(self, input_ids, attention_mask, userId, counterf):
        e1 = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :] 
        e1 = self.bert_nw(e1)
        e2 = self.user_embedding(userId)
        e3 = self.counterf_nw(counterf)
        concat = e1*e2*e3
#         concat = torch.cat([e1, e2, e3], dim=1)
        
        out = self.final(concat)
        return out


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
# torch.cuda.set_device()

In [ ]:
torch.cuda.device_count()

In [ ]:
train["userIndex"].max(), val["userIndex"].max()

In [ ]:
len(user_id_mapping), len(users_tr)

In [ ]:
device = "cuda:2"

In [ ]:
num_users = 659892# len(user_id_mapping) #1067917, 659892
embedding_dim_users = 32
m = CommentV0(model_name, 1+num_users, embedding_dim_users)

In [ ]:
m = m.float()
m = nn.DataParallel(m, device_ids = [2,3]) ## pushed always to gpu:0 :\

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('Number of trainable parameters:', count_parameters(m))

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()

In [ ]:
m

In [ ]:
m.state_dict()

In [ ]:
m.module.user_embedding.weight.data, m.module.user_embedding.weight.shape

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

In [ ]:
import sys

In [ ]:
learning_rate = 0.0001
optimizer = optim.Adam(m.parameters(), lr=learning_rate)#, weight_decay=1e-4)
# loss = nn.BCELoss()
loss=nn.BCEWithLogitsLoss()

In [ ]:
device = "cuda:2"

In [ ]:
loss_lis_train = []
loss_lis_val = []
total_loss = 0.0
tr_ep_loss = []
val_ep_loss = []

auc1 = []
for i in range(4,8):
    m.train()
    step = 0
    ep_loss = 0.0
    for batch in tqdm(train_dataloader):
        m.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        userIndex = batch[2].to(device)
        feats = batch[3].to(device)
        label = batch[4].to(device)
        
        output = m(input_ids.squeeze(1), attention_mask.squeeze(1), userIndex, feats.float())
        loss_ = loss(output, label.unsqueeze(1).float())
        loss_.backward()    
        bloss_item = loss_.item()
        loss_lis_train.append(bloss_item)
        ep_loss += bloss_item
        step += 1
        optimizer.step()
        
        
#         with autocast():
#             output =  m(input_ids.squeeze(1), attention_mask.squeeze(1), userIndex, feats) #.float())
#             loss_ = loss(output, label.unsqueeze(1).to(float))
#             loss_value = loss_.item()
#         print(step, loss_value)
#         scaler.scale(loss_).backward()
#         torch.nn.utils.clip_grad_norm_(m.parameters(), max_norm=1.0)
#         scaler.step(optimizer)
#         scaler.update()
#         loss_lis_train.append(loss_value)
#         step += 1
#         ep_loss += loss_value
#         if torch.isnan(loss_).any():
#             print("NaN loss encountered!")
#             sys.exit(1)
        
    print(f"for ep: {i} train-loss:{ep_loss/step}")
    tr_ep_loss.append(ep_loss/step)
    
    with torch.no_grad():
        m.eval()
        total_val_loss = 0
        val_step = 0
        true = []
        preds = []
        for batch in tqdm(val_dataloader):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            userIndex = batch[2].to(device)
            feats = batch[3].to(device)
            label = batch[4].to(device)
            output =  m(input_ids.squeeze(1), attention_mask.squeeze(1), userIndex, feats.float()) #input_ids.squeeze(1), attention_mask.squeeze(1)
            loss_ = loss(output, label.unsqueeze(1).float())
            val_step += 1
            loss_lis_val.append( loss_.item())
            total_val_loss += loss_lis_val[-1]
            preds.extend(output.squeeze(1).cpu().tolist())
            true.extend(label.cpu().tolist())
        print(f"for ep: {i} val-loss:{total_val_loss/val_step}")
        val_ep_loss.append(total_val_loss/val_step)
        
        precision, recall, thresholds = precision_recall_curve(true, preds)
        pr_auc = auc(recall, precision)
        roc = roc_auc_score(true, preds)
        print(f"ep:{i} roc auc : {roc}, pr auc: {pr_auc}")
        auc1.append(roc)
    torch.save(m.state_dict(), 'data/run2/my_model_ep' + str(i) +'.pth')
